**Run on Google Colab (Quickstart)**

```bash
! git clone --branch main --single-branch https://github.com/sbaaihamza/scrapping-lib.git
%cd scrapping-lib
! pip install -e ".[browser,dev]"
! playwright install
# Preferred (installs OS deps automatically on supported distros):
! playwright install --with-deps chromium
# If needed (manual deps fallback):
! apt-get update
! apt-get install -y libxcomposite1 libxcursor1 libgtk-3-0 libatk1.0-0 libcairo2 libgdk-pixbuf2.0-0
%cd /content/scrapping-lib/notebooks
```

*Note: Playwright has both sync and async APIs. These notebooks are designed to be async-safe for Jupyter/Colab. If you encounter OS dependency issues, use the `playwright install --with-deps chromium` command.*



# Scrapping Library: Capabilities & Methodology

Welcome to the `scrapping` library. This notebook is a **hands-on workshop** that teaches you the full extent of the library's capabilities and the recommended **Methodology Ladder** for building robust, compliant scrapers.

## 0) What you will learn (outcomes)

*   **Capability Map**: Everything the package offers (engines, pipeline, recipes, QA, observability).
*   **Methodology Ladder**: The structured "Try-First to Try-Last" escalation path.
*   **Diagnostics Lab**: Hands-on analysis of "Hard Targets" using the `diagnostics` module.
*   **Compliance-First Guardrails**: How to detect challenges and stop gracefully.
*   **Maintenance Loop**: Turning online exploration into stable offline fixtures and tests.

## 1) Capability Map

The library is designed to balance speed, complexity, and reliability.

| Feature | Cost/Complexity | failure modes | Artifacts to Inspect | Notebook |
| :--- | :--- | :--- | :--- | :--- |
| **HTTP Engine** | Low | Missing JS content, 403/429. | raw HTML. | 03 |
| **Browser Engine** | High | Slow, resource heavy, binaries missing. | Screenshots, HTML, Traces. | 04 |
| **Hybrid Engine** | Medium | Complexity in routing. | Combined traces. | 05 |
| **Paging** | Low-Medium | Missing items on deep pages. | URL lists. | 07 |
| **Actions DSL** | Medium | Broken selectors, timeouts. | Screenshot on error. | 04 |
| **QA Filters** | Low | Too strict (drops good data). | `rejected.jsonl`. | 01 |
| **Recipes** | High | State corruption. | `state.json`. | 06 |
| **Observability** | Low | Misconfigured loggers. | `run_report.json`. | 01 |

## 2) Methodology Ladder (Try-First → Try-Last)

Escalate your strategy only when a specific signal triggers the need.

### Escalation Flow
1.  **HTTP Minimal**: Start here. If `Ctrl+U` in browser shows the data, use `HttpEngine`.
2.  **Robust Extraction**: Use CSS/Regex + `normalize_url` to handle fragments and tracking params.
3.  **Paging**: Add pagination templates (e.g., `{page}`) if you need more than one page.
4.  **QA + Dedupe**: Add Pydantic schemas and URL/Content deduping for data quality.
5.  **Rate Limiting**: Add `rps` and `min_delay_s` to avoid 429 errors.
6.  **Browser Engine**: Switch here if content only appears after JS execution.
7.  **Hybrid Engine**: Use HTTP for fast listings and Browser for detail rendering.
8.  **Recipes**: Wrap in a Recipe if you need linear phases and resumability.
9.  **Fixtures**: Capture HTML fixtures once the scraper works to prevent future drift.

## 3) Setup & Diagnostics Lab

In this section, we use the `diagnostics` module to practice professional diagnostics against "Hard Targets".

### Helper: Resolving Run Reports

The Orchestrator might return the full report object or just paths to it. We use this helper to ensure we can always access source results.

In [ ]:
def resolve_run_report(out):
    """Robustly load run_report.json from orchestrator output."""
    if isinstance(out, dict) and "sources" in out and isinstance(out["sources"], list):
        # If it's already a full report dict with list of sources
        return out
    
    if isinstance(out, dict) and "run_report_path" in out:
        with open(out["run_report_path"]) as f:
            return json.load(f)
            
    if isinstance(out, dict) and "run_dir" in out:
        report_path = Path(out["run_dir"]) / "run_report.json"
        if report_path.exists():
            with open(report_path) as f:
                return json.load(f)
    
    print(f"Warning: Could not resolve report automatically. Output shape: {list(out.keys()) if isinstance(out, dict) else type(out)}")
    return out

In [ ]:
import json
import os
import sys
from pathlib import Path

import pandas as pd

from scrapping.diagnostics.classifiers import diagnose_http_response, recommend_next_step
from scrapping.orchestrator import Orchestrator, OrchestratorOptions

REPO_ROOT = Path.cwd().parent
sys.path.append(str(REPO_ROOT))
os.chdir(str(REPO_ROOT))

ONLINE = os.getenv('ONLINE', '0') == '1'
HARD_TARGETS = os.getenv('HARD_TARGETS', '0') == '1'

print(f'Online mode: {ONLINE} | Hard Targets: {HARD_TARGETS}')

### 3.1 Standardized Diagnostics

The `diagnostics` module helps you decide the next step based on the response.

In [ ]:
scenarios = [
    (429, {}, "Too many requests"),
    (200, {}, "Enable JavaScript to continue"),
    (200, {}, "Cloudflare Turnstile verification"),
    (403, {}, "Access Denied")
]

results = []
for status, headers, text in scenarios:
    diag = diagnose_http_response(status, headers, text)
    results.append({
        "Label": diag.label.value, 
        "Reason": diag.reason,
        "Recommendation": recommend_next_step(diag)
    })

pd.DataFrame(results)

### 3.2 Bot-Detection Diagnostics

We test our engine against bot-detectors to analyze which signals (like `webdriver` or `headless`) are flagged.

In [ ]:
config_path = 'examples/configs/real/hard_targets/browser_bot_diagnostics.json'
with open(config_path) as f:
    cfg = json.load(f)

if ONLINE and HARD_TARGETS:
    orch = Orchestrator(options=OrchestratorOptions(results_dir='results/hard_targets'))
    out = orch.run(cfg)
    
    report = resolve_run_report(out)
    summary = []
    
    # Handle both dict and list sources depending on report version
    sources = report.get('sources', [])
    if isinstance(sources, list):
        for s_res in sources:
            summary.append({
                "Target": s_res.get('source_id'), 
                "Diagnosis": s_res.get('diagnosis', {}).get('label'),
                "Recommendation": s_res.get('diagnosis', {}).get('recommendation'),
                "Artifacts": s_res.get('artifacts', {}).get('raw_detail_parts', [])[:1]
            })
    
    print("--- Hard Targets Diagnostics Lab Results ---")
    print(pd.DataFrame(summary))
else:
    print("OFFLINE: Diagnostics lab skipped. Enable ONLINE=1 and HARD_TARGETS=1 for live analysis.")

### 3.3 Challenge Detection (Compliance-First)

**Policy**: Success == Detection + Graceful Stop. Do not attempt bypass.

**Important Note**: Cloudflare explicitly states that automated testing suites (like Playwright/Selenium) are often detected as bots by Turnstile. This is the intended behavior of such security measures. Our goal is professional diagnosis and safe behavior: detecting the wall, recording the event, and stopping execution to avoid further escalation or account flagging.

In [ ]:
from scrapping.diagnostics.classifiers import diagnose_rendered_dom

html_fixture = "<html><body><div id='turnstile-widget'></div></body></html>"
diag = diagnose_rendered_dom(html_fixture)

print(f"Target: Turnstile Demo | Label: {diag.label.value} | Action: {recommend_next_step(diag)}")

## 4) Maintenance Loop: Exploration to Regression

Professional scraping requires locking in successes as offline tests.

1.  **Diagnose**: Ensure the page loads correctly and content is present.
2.  **Capture**: Use the CLI to save the HTML as a fixture.
    ```bash
    scrap capture-fixture --url https://example.com --out tests/fixtures/html/example.html
    ```
3.  **Scaffold**: Generate a regression test asserting extraction works on that fixture.
    ```bash
    scrap scaffold-test --fixture tests/fixtures/html/example.html --extract css --pattern 'h1' --expect-count 1
    ```
4.  **Commit**: Push the fixture and test to your repository.

## 5) Minimal Runnable Examples (Offline)

### 5.1 HTTP Minimal (books.toscrape.com)

In [ ]:
from scrapping.engines.http import HttpEngine

engine = HttpEngine()
url = "https://books.toscrape.com/"
if ONLINE:
    res = engine.get(url)
    print(f"Fetch OK: {res.ok} | status: {res.status_code}")
else:
    print("OFFLINE: Skipping live fetch.")

### Final Checklist for New Scrapers
- [ ] Did I try **HTTP first**?
- [ ] Did the **Diagnosis** confirm it's safe to proceed?
- [ ] Have I added **rate limiting** (politeness)?
- [ ] Have I saved an **offline fixture** for regression testing?

### Final Checklist for Advanced / Hard Targets
- [ ] OS dependencies verified (`playwright install --with-deps chromium`).
- [ ] Diagnostics label exists for every failure mode.
- [ ] Stop condition implemented for challenges (no infinite retries).
- [ ] Artifacts captured on every failure (HTML + Screenshot).
- [ ] Fixture captured + regression test scaffolded.
- [ ] Idempotency verified (no duplicates in output).
- [ ] Observability review completed (`run_report.json` checked).